In [73]:
import numpy as np
from scipy.sparse import load_npz, lil_matrix, identity, csr_matrix
from sklearn.preprocessing import normalize
import gzip
import json

sl = load_npz('data/graph4sparse_1.npz')
print(sl.count_nonzero(), sl.max(), sl.min())


1162091760 1.0 0.0


In [ ]:
fi = gzip.open('data/wikidata_translation_v1.tsv.gz', 'rb')
j = json.load(open(f'data/label4sparse.json'))
QUS = int(j['maxq'])

bigq = np.zeros((QUS, 200), dtype=np.half)
print(fi.readline())
uk = 0
for l in fi:
    l = l.decode().split('\t')
    if '<http://www.wikidata.org/entity/Q' in l[0]:
        qid = int(l[0].replace('<http://www.wikidata.org/entity/Q', '').replace('>', ''))
        bigq[qid] = [np.half(a) for a in l[1:]]
        uk += 1
        
print(uk)

b'78404883\t4151\t200\n'


In [ ]:
np.savez_compressed('data/bigq', vectors=bigq)

In [45]:
qids = ['Q28865', 'Q271218', 'Q10811', 'Q2407', 'Q842014', 'Q21198', 'Q420', 'Q80006', 'Q460387', 'Q179012']
desc = ['Python language', 'Python snake', 'Reptils', 'C++', 'Django Python', 'Computer Science', 'Biology', 'Computer Programming', 'Scrum', 'Project management']

qs = [int(a[1:]) for a in qids]

In [57]:
qrank = np.zeros((1152625061), int)
import gzip
fi = gzip.open('/backup/wikidata/qrank.csv.gz', 'rb')
fi.readline()
uk = 0
for l in fi:
    q, r = l.decode().strip('\n').split(',')
    qrank[int(q[1:])] = int(r)
    uk += 1
print(uk), print([qrank[int(q[1:])] for q in qids])

28990031

In [70]:
qrankl = np.log(qrank+.0001)
qrankl = np.cast(qrankl / qrankl.max(), np.half)

TypeError: '_typedict' object is not callable

In [69]:
#%%timeit
#sl[qs].todense()[:,qs]
#m = normalize(sl[qs], norm='l2', axis=1)
m = sl[qs]
slic = m*m.T
slic.data = np.minimum(slic.data / 5., 1)

slic = (slic / slic.max()) + m.todense()[:,qs]
#slic = m.todense()[:,qs]

for i in range(len(qs)):
    for j in range(len(qs)):
        if i <= j:
            print(desc[i], desc[j], qrank[qs[j]], qrankl[qs[j]], round(slic[i, j], 2))
    

Python language Python language 13740291 0.8556477584951567 2.0
Python language Python snake 272866 0.6516195842311826 0.0
Python language Reptils 2900452 0.7746704158309173 0.01
Python language C++ 3650 0.427020125032955 1.7
Python language Django Python 639953 0.6959961364416117 0.14
Python language Computer Science 3131053 0.7786531403258956 0.3
Python language Biology 4208329 0.7940470489239255 0.06
Python language Computer Programming 1887097 0.7522936848070895 0.56
Python language Scrum 2065137 0.756987240720752 0.16
Python language Project management 1453877 0.7387161656483875 0.05
Python snake Python snake 272866 0.6516195842311826 1.66
Python snake Reptils 2900452 0.7746704158309173 0.18
Python snake C++ 3650 0.427020125032955 0.0
Python snake Django Python 639953 0.6959961364416117 0.0
Python snake Computer Science 3131053 0.7786531403258956 0.0
Python snake Biology 4208329 0.7940470489239255 0.01
Python snake Computer Programming 1887097 0.7522936848070895 0.0
Python snake S

In [41]:
m = csr_matrix([[0,0,0],[1,0,0],[0,1,0],])
m = m*m*0.8
m.todense()

matrix([[0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0.8, 0. , 0. ]])